# Chronos Forecaster Demo

This notebook demonstrates the simple usage of the `chronos_forecaster` package with both engines:
- **Chronos-Bolt**: Fast zero-shot forecasting
- **Chronos-2**: Advanced forecasting with covariate support

## 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
from chronos_forecaster import ChronosForecaster
import warnings
warnings.filterwarnings('ignore')

## 2. Generate Sample Data

In [2]:
# Create synthetic time series data
np.random.seed(42)
dates = pd.date_range(start='2024-01-01', periods=200, freq='D')
trend = np.linspace(100, 150, 200)
seasonal = 10 * np.sin(np.linspace(0, 8*np.pi, 200))
noise = np.random.normal(0, 3, 200)
values = trend + seasonal + noise

df = pd.DataFrame({
    'date': dates,
    'target': values
})

print(f"Generated {len(df)} data points")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
display(df)

Generated 200 data points
Date range: 2024-01-01 00:00:00 to 2024-07-18 00:00:00


,date,target
0,2024-01-01,101.490142
1,2024-01-02,101.096060
2,2024-01-03,104.944708
3,2024-01-04,109.021711
4,2024-01-05,105.142221
...,...,...
195,2024-07-14,145.311271
196,2024-07-15,142.895806
197,2024-07-16,147.459533
198,2024-07-17,148.663773


## 3. Chronos-Bolt Predictions (Fast Zero-Shot)

In [3]:
# Create Chronos-Bolt forecaster
forecaster_bolt = ChronosForecaster(
    forecast_horizon=14,
    datetime_col='date',
    target_col='target',
    frequency='D',
    engine='chronos'
)

# Generate predictions
predictions_bolt = forecaster_bolt.predict(df)

# Print predictions
print("Chronos-Bolt Predictions:")
display(predictions_bolt)

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Chronos-Bolt Predictions:


,date,target_predicted,lower_bound,upper_bound
0,2024-07-19,149.742355,145.970398,153.234116
1,2024-07-20,151.278091,147.498276,154.700378
2,2024-07-21,152.168427,148.621979,155.585724
3,2024-07-22,153.558960,149.976105,156.953156
4,2024-07-23,154.341309,150.755783,157.758820
5,2024-07-24,154.928162,151.246765,158.368164
6,2024-07-25,155.436035,151.792480,159.024612
7,2024-07-26,156.874298,153.112335,160.570099
8,2024-07-27,157.972321,154.271255,161.676300
9,2024-07-28,159.329590,155.387711,163.146591


## 4. Chronos-2 Predictions (Zero-Shot)

In [4]:
# Create Chronos-2 forecaster
forecaster_c2 = ChronosForecaster(
    forecast_horizon=14,
    datetime_col='date',
    target_col='target',
    frequency='D',
    engine='chronos2'
)

# Generate predictions
predictions_c2 = forecaster_c2.predict(df)

# Print predictions
print("Chronos-2 Predictions:")
display(predictions_c2)

Chronos-2 Predictions:


,date,target_predicted,lower_bound,upper_bound
0,2024-07-19,150.804062,147.815414,154.160629
1,2024-07-20,152.592026,149.506195,155.652527
2,2024-07-21,154.256729,151.120056,157.452087
3,2024-07-22,155.564484,152.370361,158.552826
4,2024-07-23,156.537567,153.373993,159.855988
5,2024-07-24,157.263428,154.091736,160.566559
6,2024-07-25,158.371582,155.236923,161.711212
7,2024-07-26,159.507629,156.077011,162.875320
8,2024-07-27,160.799088,157.478973,164.073471
9,2024-07-28,161.925797,158.430359,165.343307


## 5. Chronos-2 with Past Covariates Only

In [5]:
# Create past covariates (historical features that align with the target data)
past_cov_df = pd.DataFrame({
    'date': dates,
    'temperature': 20 + 10 * np.sin(np.linspace(0, 8*np.pi, 200)) + np.random.normal(0, 2, 200),
    'day_of_week': dates.dayofweek,
    'is_weekend': dates.dayofweek.isin([5, 6]).astype(int)
})

print("Past Covariates Sample:")
display(past_cov_df)

# Generate predictions with past covariates
predictions_c2_past = forecaster_c2.predict(
    df=df,
    past_covariates_df=past_cov_df
)

print("\nChronos-2 Predictions with Past Covariates:")
display(predictions_c2_past)

Past Covariates Sample:


,date,temperature,day_of_week,is_weekend
0,2024-01-01,20.715575,0,0
1,2024-01-02,22.381166,1,0
2,2024-01-03,24.665232,2,0
3,2024-01-04,25.806457,3,0
4,2024-01-05,22.084317,4,0
...,...,...,...,...
195,2024-07-14,14.221993,6,1
196,2024-07-15,12.874878,0,0
197,2024-07-16,20.208615,1,0
198,2024-07-17,18.511323,2,0



Chronos-2 Predictions with Past Covariates:


,date,target_predicted,lower_bound,upper_bound
0,2024-07-19,150.869690,147.797668,154.097137
1,2024-07-20,152.462250,149.291138,155.461365
2,2024-07-21,153.964752,150.759583,157.102036
3,2024-07-22,155.191589,152.047226,158.138489
4,2024-07-23,156.263107,152.994476,159.414688
5,2024-07-24,156.898010,153.487549,160.083252
6,2024-07-25,157.943390,154.661057,161.133118
7,2024-07-26,158.898499,155.256592,162.113464
8,2024-07-27,160.086182,156.500046,163.219818
9,2024-07-28,161.163361,157.537811,164.515625


## 6. Chronos-2 with Past AND Future Covariates

In [6]:
# Create future covariates (known future values for the forecast horizon)
# These should be features you actually know ahead of time (e.g., calendar features, weather forecasts)
from datetime import timedelta

future_dates = pd.date_range(start=dates[-1] + timedelta(days=1), periods=14, freq='D')
future_cov_df = pd.DataFrame({
    'date': future_dates,
    'temperature': 20 + 10 * np.sin(np.linspace(8*np.pi, 8*np.pi + 0.7*np.pi, 14)),  # Simulated forecast
    'day_of_week': future_dates.dayofweek,
    'is_weekend': future_dates.dayofweek.isin([5, 6]).astype(int)
})

print("Future Covariates Sample:")
display(future_cov_df)

# Generate predictions with both past and future covariates
predictions_c2_both = forecaster_c2.predict(
    df=df,
    past_covariates_df=past_cov_df,
    future_covariates_df=future_cov_df
)

print("\nChronos-2 Predictions with Past + Future Covariates:")
display(predictions_c2_both)

Future Covariates Sample:


,date,temperature,day_of_week,is_weekend
0,2024-07-19,20.000000,4,0
1,2024-07-20,21.683570,5,1
2,2024-07-21,23.319079,6,1
3,2024-07-22,24.859834,0,0
4,2024-07-23,26.261852,1,0
5,2024-07-24,27.485107,2,0
6,2024-07-25,28.494679,3,0
7,2024-07-26,29.261746,4,0
8,2024-07-27,29.764411,5,1
9,2024-07-28,29.988322,6,1



Chronos-2 Predictions with Past + Future Covariates:


,date,target_predicted,lower_bound,upper_bound
0,2024-07-19,150.891785,147.923370,153.959717
1,2024-07-20,152.429474,149.458633,155.261688
2,2024-07-21,153.970978,150.873932,156.948349
3,2024-07-22,155.222046,152.178375,158.051178
4,2024-07-23,156.237289,153.084076,159.287949
5,2024-07-24,156.844162,153.604797,159.948181
6,2024-07-25,157.989258,154.846802,161.095215
7,2024-07-26,158.945786,155.446838,162.099243
8,2024-07-27,160.126434,156.796722,163.156158
9,2024-07-28,161.070282,157.675873,164.342789


## Summary

**Covariate Types:**
- **Past Covariates**: Historical features aligned with your target data (e.g., past temperature, past sales)
- **Future Covariates**: Features you know ahead of time (e.g., day of week, holidays, weather forecasts)

**Key Points:**
- Past covariates help the model understand historical patterns
- Future covariates provide known information about the forecast period
- Only Chronos-2 supports covariates (Chronos-Bolt does not)
- Future covariates must truly be known in advance to avoid data leakage